## Spanish NLP Libraries
- tokenizer, customizing
- noun extraction, POS, lemma
- ngrams
- stopwords

In [1]:
import spacy
from spacy.lang.es.examples import sentences

import nltk.data
from nltk.tokenize import word_tokenize, WordPunctTokenizer
nltk.download('punkt', quiet=True)

import textacy
from textacy.extract import ngrams, entities

from urllib.request import urlopen
from bs4 import BeautifulSoup

### Tokenize
1. spacy (https://spacy.io/models/es)
4. textacy (https://textacy.readthedocs.io/en/latest/quickstart.html#working-with-many-languages)
2. NLTK spanish pickle, wordpunct
3. Customary

#### 1. Spacy

In [2]:
# sample text
sentences

['Apple está buscando comprar una startup del Reino Unido por mil millones de dólares.',
 'Los coches autónomos delegan la responsabilidad del seguro en sus fabricantes.',
 'San Francisco analiza prohibir los robots delivery.',
 'Londres es una gran ciudad del Reino Unido.',
 'El gato come pescado.',
 'Veo al hombre con el telescopio.',
 'La araña come moscas.',
 'El pingüino incuba en su nido sobre el hielo.',
 '¿Dónde estais?',
 '¿Quién es el presidente Francés?',
 '¿Dónde está encuentra la capital de Argentina?',
 '¿Cuándo nació José de San Martín?']

In [3]:
nlp = spacy.load("es_core_news_sm")
doc = nlp(sentences[3]) # lower의 옵션도 고려할 수 있다.
print(doc.text)

Londres es una gran ciudad del Reino Unido.


In [4]:
# tokenize
spacy_tokens = [token for token in doc]
sorted(spacy_tokens)

[Londres, es, una, gran, ciudad, del, Reino, Unido, .]

#### 2. Textacy
- Since spaCy’s pipelines are language-dependent, we have to load a particular pipeline to match the text; when working with texts from multiple languages, this can be a pain. Fortunately, textacy includes automatic language detection to apply the right pipeline to the text, and it caches the loaded language data to minimize wait time and hassle. 
- 버전마다, 언어마다 되는 게 상이함

In [5]:
ttc_nlp = textacy.load_spacy_lang("es_core_news_sm")
ttc_doc = textacy.make_spacy_doc(sentences[3], lang=ttc_nlp)
ttc_doc

Londres es una gran ciudad del Reino Unido.

In [6]:
ttc_tokens = [token for token in ttc_doc]
sorted(ttc_tokens)

[Londres, es, una, gran, ciudad, del, Reino, Unido, .]

In [7]:
ttc_doc._.preview # summary 역할

'Doc(9 tokens: "Londres es una gran ciudad del Reino Unido.")'

#### 3. NLTK
- pickle: sentence splitter
- WordPunct

In [8]:
nltk_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

In [9]:
nltk_tokenizer.tokenize(sentences[3]) # 한 문장이므로

['Londres es una gran ciudad del Reino Unido.']

In [10]:
nltk_tokenizer.tokenize(' '.join(sentences))

['Apple está buscando comprar una startup del Reino Unido por mil millones de dólares.',
 'Los coches autónomos delegan la responsabilidad del seguro en sus fabricantes.',
 'San Francisco analiza prohibir los robots delivery.',
 'Londres es una gran ciudad del Reino Unido.',
 'El gato come pescado.',
 'Veo al hombre con el telescopio.',
 'La araña come moscas.',
 'El pingüino incuba en su nido sobre el hielo.',
 '¿Dónde estais?',
 '¿Quién es el presidente Francés?',
 '¿Dónde está encuentra la capital de Argentina?',
 '¿Cuándo nació José de San Martín?']

In [11]:
# wordpunct
wp_tokenizer = WordPunctTokenizer()
wp_tokenizer.tokenize(sentences[-1])

['¿', 'Cuándo', 'nació', 'José', 'de', 'San', 'Martín', '?']

In [12]:
# word_tokenize와 비교 
word_tokenize(sentences[-1]) # 영어에 없는 기호는 인식하지 못함.

['¿Cuándo', 'nació', 'José', 'de', 'San', 'Martín', '?']

#### 4. customizing
내가 원하는 대로 토크나이저 만들기

In [13]:
def treat_del(word:str):
    ''' 
    del -> de, el
    al -> a, el
    '''
    assert word.lower() in ['del', 'al'], 'wrong word'
    word = word[:-1]
    
    return [word, 'el']
    
def a_tokenizer(sentence:str):
    tokenizer = nltk.tokenize.WordPunctTokenizer()
    tokens = tokenizer.tokenize(sentence.lower())
    
    for t in tokens:
        if t in ['del', 'al']:
            new =  treat_del(t)
            tokens.extend(new)
            tokens.remove(t)
    return tokens

def b_tokenizer(sentence:str):
    nlp = spacy.load("es_core_news_sm")
    tokens = [str(token) for token in nlp(sentence.lower())]
    
    for t in tokens:
        if t in ['del', 'al']:
            new =  treat_del(t)
            tokens.extend(new)
            tokens.remove(t)
    return tokens

def ab_tokenizer(sentence:str, tokenizer=WordPunctTokenizer()):
    # 토크나이저에 따른 토큰 방식
    sentence = sentence.lower()
    tokenizer_name = str(tokenizer).lower()
    
    if tokenizer_name == 'nlp':
        nlp = spacy.load("es_core_news_sm")
        tokens = [str(token) for token in nlp(sentence)]
    elif 'word_tokenize' in tokenizer_name.split():
        tokens = tokenizer(sentence)
    else:  # nltk의 다른 모든 토크나이저
        tokens = tokenizer.tokenize(sentence)
    
    for t in tokens:
        if t in ['del', 'al']:
            new =  treat_del(t)
            tokens.extend(new)
            tokens.remove(t)
    return tokens

In [14]:
for a, b, c in zip(a_tokenizer(sentences[0]),
                   b_tokenizer(sentences[0]),
                   ab_tokenizer(sentences[0], nltk.tokenize.word_tokenize)):
    print('{0:^20}{1:^20}{2:^20}'.format(a, b, c))

       apple               apple               apple        
        está                está                está        
      buscando            buscando            buscando      
      comprar             comprar             comprar       
        una                 una                 una         
      startup             startup             startup       
       reino               reino               reino        
       unido               unido               unido        
        por                 por                 por         
        mil                 mil                 mil         
      millones            millones            millones      
         de                  de                  de         
      dólares             dólares             dólares       
         .                   .                   .          
         de                  de                  de         
         el                  el                  el         


In [15]:
a_tokenizer(sentences[0])

['apple',
 'está',
 'buscando',
 'comprar',
 'una',
 'startup',
 'reino',
 'unido',
 'por',
 'mil',
 'millones',
 'de',
 'dólares',
 '.',
 'de',
 'el']

### noun extraction, POS, lemma
- 명사 추출
- POS (part-of-speeech)
- lemma: 표제어

#### spacy로 명사 추출

In [16]:
doc = nlp(sentences[1].lower())
test = list(doc.noun_chunks)
print('original= ', doc)
print('nouns= ', test) # 관사도 같이 추출됨을 알 수 있다.

original=  los coches autónomos delegan la responsabilidad del seguro en sus fabricantes.
nouns=  [los coches, la responsabilidad, seguro, sus fabricantes]


In [17]:
# 전체 단어의 POS
[(token.text, token.pos_) for token in doc]

[('los', 'DET'),
 ('coches', 'NOUN'),
 ('autónomos', 'ADJ'),
 ('delegan', 'VERB'),
 ('la', 'DET'),
 ('responsabilidad', 'NOUN'),
 ('del', 'ADP'),
 ('seguro', 'NOUN'),
 ('en', 'ADP'),
 ('sus', 'DET'),
 ('fabricantes', 'NOUN'),
 ('.', 'PUNCT')]

In [18]:
# 전체 단어의 표제어 추출
[(token.text, token.lemma_) for token in doc]

[('los', 'el'),
 ('coches', 'coche'),
 ('autónomos', 'autónomo'),
 ('delegan', 'delegar'),
 ('la', 'el'),
 ('responsabilidad', 'responsabilidad'),
 ('del', 'del'),
 ('seguro', 'seguro'),
 ('en', 'en'),
 ('sus', 'su'),
 ('fabricantes', 'fabricante'),
 ('.', '.')]

In [19]:
# 추출된 명사의 (단어, POS, 레마)
[(np, np.root.tag_, np.root.lemma_) for np in test] 

[(los coches, 'NOUN', 'coche'),
 (la responsabilidad, 'NOUN', 'responsabilidad'),
 (seguro, 'NOUN', 'seguro'),
 (sus fabricantes, 'NOUN', 'fabricante')]

#### textacy의 부가적 기능
- textstats: compute various basic and readability statistics
- text_utils.keyword_in_context: 해당 단어가 나타나는 문맥

In [20]:
ttc_doc

Londres es una gran ciudad del Reino Unido.

In [21]:
text_stats = textacy.TextStats(ttc_doc)

In [22]:
# (단어 수, 음절 수, 알파벳 수)
text_stats.n_words, text_stats.n_syllables, text_stats.n_chars 

(8, 11, 35)

In [23]:
# 엔트로피: 문장이 얼마나 많은 정보를 갖고 있는가
text_stats.entropy

3.0

In [24]:
# 가독성 (1-100)
text_stats.flesch_kincaid_grade_level, text_stats.flesch_reading_ease

(3.7550000000000026, 116.17500000000001)

In [25]:
# 문맥 정보 (다량의 코퍼스에서 유용)
list(textacy.text_utils.keyword_in_context(ttc_doc.text, 'gran', window_width=20)) # 주변 몇 글자를 볼 것인가

     Londres es una  gran  ciudad del Reino Un


[]

### ngrams
[Londres es], [es una], [una gran] ...
- 앞뒤 n개의 단어 정보를 알려주는 기능

In [26]:
from textacy.extract import ngrams

In [27]:
list(ngrams(ttc_doc, 2,
            filter_stops = False, # True=마지막만 출력
           filter_punct = False, # True=문장부호 붙은 단어 제외
           filter_nums = False,
           min_freq = 1)) # 최소 빈도수

[Londres es,
 es una,
 una gran,
 gran ciudad,
 ciudad del,
 del Reino,
 Reino Unido,
 Unido.]

In [28]:
# 간단하게는
list(ngrams(ttc_doc, 3, filter_stops=False))

[Londres es una,
 es una gran,
 una gran ciudad,
 gran ciudad del,
 ciudad del Reino,
 del Reino Unido]

### stopwords | 불용어
자주 쓰이지만 중요한 의미를 내포하지 않으므로 주로 제거함.

In [29]:
import nltk
from nltk.corpus import stopwords

In [30]:
stop_words = stopwords.words('spanish')
print(stop_words)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [31]:
len(stop_words)

313

In [32]:
sentences[0]

'Apple está buscando comprar una startup del Reino Unido por mil millones de dólares.'

In [33]:
# tokenize -> stopwords 제거
[w for w in word_tokenize(sentences[0]) if w.lower() not in stop_words]

['Apple',
 'buscando',
 'comprar',
 'startup',
 'Reino',
 'Unido',
 'mil',
 'millones',
 'dólares',
 '.']

In [34]:
' '.join(w for w in word_tokenize(sentences[0]) if w.lower() not in stop_words)

'Apple buscando comprar startup Reino Unido mil millones dólares .'

### Summary
작업의 목표:
- 코퍼스에 대한 이해
- 원하는 테스크에 맞는 단어 선별 (중요한 단어만 남기기 위함)